# Revista Mineira de Contabilidade – RMC

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    h2_results = soup.find_all('h2', {'class': 'title'})
    a_links = []
    for item in h2_results:
        try:
            if 'Editorial' not in item.find('a').text:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://revista.crcmg.org.br/rmc/search/search?query=__query__&searchJournal=1&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2015&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://revista.crcmg.org.br/rmc/article/view/704'

In [6]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [8]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split('-')[0]
    afiliacao = afiliacao.split('(')[0]
    return afiliacao.strip()

In [9]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [10]:
captura_titulo(soup)

{'Título': 'CONTABILIDADE AMBIENTAL: UM ESTUDO SOBRE A PERCEPÇÃO DOS ESTUDANTES DE GRADUAÇÃO EM CIÊNCIAS CONTÁBEIS DA CIDADE DE MACEIÓ/AL'}

In [11]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [12]:
captura_data(soup)

{'Data de Publicação': '2017-11-30'}

In [13]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract_ps = abstract_section.find_all('p')
        abstract = abstract_ps[0].text
        abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [14]:
captura_resumo(soup)

{'Resumo': 'O objetivo deste artigo foi analisar o nível de conhecimento de discentes de cursos de graduação em ciências contábeis em relação a tópicos correlatos à contabilidade ambiental. A metodologia utilizada quanto ao objetivo proposto se caracteriza como descritiva e, quanto aos procedimentos metodológicos, trata-se de um levantamento (survey). A amostra investigada, denominada como não probabilística intencional, foi determinada considerando as instituições de ensino superior (IES), presentes na cidade de Maceió (AL), que oferecem o curso de ciências contábeis e obtiveram aprovação no Exame Nacional de Desenvolvimento de Estudante (ENADE) no ano de 2014. O total de respondentes foi de 165 discentes de seis IES. A coleta de dados foi feita com a aplicação de questionário, sendo tratados por meio de estatística descritiva, na sua dimensão quantitativa. Os resultados evidenciaram que o nível de conhecimento dos discentes a respeito dos tópicos correlatos à contabilidade ambiental 

In [15]:
def captura_palavras_chave(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract_ps = abstract_section.find_all('p')
        kw = abstract_ps[2].text
        kw = kw.replace('Palavras-chave:', '').replace(',', '.').replace(';', '.')
        kw = ' '.join(kw.split())
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [16]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Ciências Contábeis. Contabilidade Ambiental. Gestão Ambiental.'}

In [17]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [18]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Gesualdo Menezes Cavalcante',
 'Autor 2': 'Sheila Mirian Barbosa Israel',
 'Autor 3': 'Márcio Mateus Ferreira de Aquino',
 'Autor 4': 'Alessandra Carla Ceolin',
 'Autor 5': '',
 'Autor 6': ''}

In [19]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [20]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Federal Rural de Pernambuco',
 'Afiliação 2': 'Universidade Federal Rural de Pernambuco',
 'Afiliação 3': 'Universidade Federal Rural de Pernambuco',
 'Afiliação 4': 'Universidade Federal Rural de Pernambuco',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [21]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista Mineira de Contabilidade – RMC'}
    url_search_text = 'https://revista.crcmg.org.br/rmc/search/search?query=__query__&searchJournal=1&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2015&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [22]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head()

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2017-11-30,CONTABILIDADE AMBIENTAL: UM ESTUDO SOBRE A PER...,O objetivo deste artigo foi analisar o nível d...,Ciências Contábeis. Contabilidade Ambiental. G...,4,Gesualdo Menezes Cavalcante,Sheila Mirian Barbosa Israel,Márcio Mateus Ferreira de Aquino,Alessandra Carla Ceolin,,,Universidade Federal Rural de Pernambuco,Universidade Federal Rural de Pernambuco,Universidade Federal Rural de Pernambuco,Universidade Federal Rural de Pernambuco,,,https://revista.crcmg.org.br/rmc/article/view/704,Revista Mineira de Contabilidade – RMC
1,2016-08-08,A INTRODUÇÃO DA CONTABILIDADE AMBIENTAL: APLIC...,A contabilidade é um sistema de informações fo...,,2,Geová José Madeira,Glauber Lima Barbosa,,,,,,,,,,,https://revista.crcmg.org.br/rmc/article/view/425,Revista Mineira de Contabilidade – RMC
2,2018-04-20,DISCLOSURE AMBIENTAL: UM ESTUDO DAS DEMONSTRAÇ...,As emergentes discussões acerca dos processos ...,,5,Karine de Oliveira Ozio,Octavio Gomes do Carmo,Luciano Gomes dos Reis,Jaqueline dos Santos Ferrarezi,Karina Rocha Henriques Gehlen,,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina.,Universidade Estadual de Londrina,,https://revista.crcmg.org.br/rmc/article/view/710,Revista Mineira de Contabilidade – RMC
3,2016-04-26,PANORAMA DA PESQUISA CIENTÍFICA SOBRE PASSIVO ...,Este estudo buscou revelar um panorama das pes...,,3,Yarles Jean Piona,Gabriel Moreira Campos,Donizete Reina,,,,UFES,Universidade Federal do Espírito Santo,UFES,,,,https://revista.crcmg.org.br/rmc/article/view/85,Revista Mineira de Contabilidade – RMC
4,2016-07-12,Contabilidade ambiental: Evidenciação das ques...,"A preservação do meio ambiente tem sido, nas ú...",,4,Adriane Bernardo de Oliveira Moreira,Laércio Antônio Gonçalves Jacovine,Márcio Lopes da Silva,Sebastião Renato Valverde,,,,,,,,,https://revista.crcmg.org.br/rmc/article/view/365,Revista Mineira de Contabilidade – RMC


In [23]:
df.to_csv('../data/RCM.csv', index=False)

In [24]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2017-11-30,CONTABILIDADE AMBIENTAL: UM ESTUDO SOBRE A PER...,O objetivo deste artigo foi analisar o nível d...,Ciências Contábeis. Contabilidade Ambiental. G...,4,Gesualdo Menezes Cavalcante,Sheila Mirian Barbosa Israel,Márcio Mateus Ferreira de Aquino,Alessandra Carla Ceolin,,,Universidade Federal Rural de Pernambuco,Universidade Federal Rural de Pernambuco,Universidade Federal Rural de Pernambuco,Universidade Federal Rural de Pernambuco,,,https://revista.crcmg.org.br/rmc/article/view/704,Revista Mineira de Contabilidade – RMC
1,2016-08-08,A INTRODUÇÃO DA CONTABILIDADE AMBIENTAL: APLIC...,A contabilidade é um sistema de informações fo...,,2,Geová José Madeira,Glauber Lima Barbosa,,,,,,,,,,,https://revista.crcmg.org.br/rmc/article/view/425,Revista Mineira de Contabilidade – RMC
2,2018-04-20,DISCLOSURE AMBIENTAL: UM ESTUDO DAS DEMONSTRAÇ...,As emergentes discussões acerca dos processos ...,,5,Karine de Oliveira Ozio,Octavio Gomes do Carmo,Luciano Gomes dos Reis,Jaqueline dos Santos Ferrarezi,Karina Rocha Henriques Gehlen,,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina.,Universidade Estadual de Londrina,,https://revista.crcmg.org.br/rmc/article/view/710,Revista Mineira de Contabilidade – RMC
3,2016-04-26,PANORAMA DA PESQUISA CIENTÍFICA SOBRE PASSIVO ...,Este estudo buscou revelar um panorama das pes...,,3,Yarles Jean Piona,Gabriel Moreira Campos,Donizete Reina,,,,UFES,Universidade Federal do Espírito Santo,UFES,,,,https://revista.crcmg.org.br/rmc/article/view/85,Revista Mineira de Contabilidade – RMC
4,2016-07-12,Contabilidade ambiental: Evidenciação das ques...,"A preservação do meio ambiente tem sido, nas ú...",,4,Adriane Bernardo de Oliveira Moreira,Laércio Antônio Gonçalves Jacovine,Márcio Lopes da Silva,Sebastião Renato Valverde,,,,,,,,,https://revista.crcmg.org.br/rmc/article/view/365,Revista Mineira de Contabilidade – RMC
5,2016-08-09,Contabilidade Ambiental - Fatores ambientais e...,,,1,Vera Luise Becke,,,,,,,,,,,,https://revista.crcmg.org.br/rmc/article/view/454,Revista Mineira de Contabilidade – RMC
6,2016-08-08,CONTABILIDADE NA GESTÃO AMBIENTAL E SOCIAL,,,1,Adelvandro Felício de Castro,,,,,,,,,,,,https://revista.crcmg.org.br/rmc/article/view/430,Revista Mineira de Contabilidade – RMC
7,2016-11-30,Estratégias de aprendizagem autorregulada em c...,O objetivo do estudo consistiu em analisar o p...,,4,Thiago Bruno de Jesus Silva,Samuel Haag,Vania Tanira Biavatti,Luis Antonio Lay,,,FURB,Mestrando em Ciências Contábeis na Universidad...,FURB,,,,https://revista.crcmg.org.br/rmc/article/view/348,Revista Mineira de Contabilidade – RMC
8,2020-12-23,Nível de evidenciação de custos e investimento...,Este estudo tem como objetivo analisar a relaç...,,3,Fabio Henrique Voigt,João Victor Machado,Alison Martins Meurer,,,,Universidade Estadual de Ponta Grossa,Universidade Estadual de Ponta Grossa,Universidade Federal do Paraná,,,,https://revista.crcmg.org.br/rmc/article/view/...,Revista Mineira de Contabilidade – RMC
9,2016-08-16,Demonstrações Contábeis e Aspectos da Contabil...,,,1,Rosalva Pinto Braga,,,,,,,,,,,,https://revista.crcmg.org.br/rmc/article/view/535,Revista Mineira de Contabilidade – RMC
